## Score me

In [1]:
import fitz  # PyMuPDF
import pandas as pd
import re
import os

In [8]:
import fitz
import re
import pandas as pd

def extract_transactions(pdf_path):
    doc = fitz.open(pdf_path)
    all_transactions = []
    current_page = 1

    for page in doc:
        text = page.get_text()
        lines = text.split('\n')
        page_transactions = []
        
        for line in lines:
            # Match date pattern
            date_match = re.search(r'(\d{2}-[A-Za-z]{3}-\d{4})', line)
            if date_match:
                date = date_match.group(1)
                description = line[date_match.end():].strip()
                
                # Extract amounts
                amounts = re.findall(r'(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)', description)
                debit, credit, balance = "", "", ""
                
                if amounts:
                    if 'Dr' in description:
                        if len(amounts) >= 3:
                            debit, credit, balance = amounts[-3], amounts[-2], amounts[-1] + 'Dr'
                        elif len(amounts) == 2:
                            debit, balance = amounts[0], amounts[1] + 'Dr'
                        elif len(amounts) == 1:
                            balance = amounts[0] + 'Dr'
                    else:
                        if len(amounts) >= 3:
                            debit, credit, balance = amounts[-3], amounts[-2], amounts[-1] + 'Dr'
                        elif len(amounts) == 2:
                            credit, balance = amounts[0], amounts[1] + 'Dr'
                        elif len(amounts) == 1:
                            balance = amounts[0] + 'Dr'
                
                # Clean up description
                description = re.sub(r'(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)', '', description).strip()
                description = description.replace('Dr', '').strip()
                
                page_transactions.append({
                    'Page': current_page,
                    'Date': date,
                    'Description': description,
                    'Debit': debit,
                    'Credit': credit,
                    'Balance': balance
                })
        
        all_transactions.extend(page_transactions)
        current_page += 1

    doc.close()
    return all_transactions

# The save_to_excel function remains the same

# Path to your PDF file
pdf_path = "test3.pdf"
output_path = "transactions.xlsx"

# Extract transaction data
transactions = extract_transactions(pdf_path)

# Save to Excel
save_to_excel(transactions, output_path)

print("Data extraction and saving completed.")


Data extraction and saving completed.
